In [1]:
import sys

# adapt paths
sys.path.append("/home/cctrotte/krauthammer/stcs_code/pysurvival_mine/")
sys.path.append("/home/cctrotte/krauthammer/stcs_code/pysurvival_mine/AutonSurvival/")

In [2]:
#### 1 - Importing packages
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival_mine.models.simulations import SimulationModel
from pysurvival_mine.models.coxph_mine import MineNonLinearCoxPHModel
from pysurvival_mine.utils.metrics import concordance_index
from pysurvival_mine.utils.metrics import concordance_index_mine
from sklearn.preprocessing import StandardScaler
from pysurvival_mine.utils.display import integrated_brier_score

#%matplotlib inline  # To use with Jupyter notebooks

#### 2 - Generating the dataset from a nonlinear Weibull parametric model
# Initializing the simulation model
sim = SimulationModel(
    survival_distribution="weibull",
    risk_type="Gaussian",
    censored_parameter=2.1,
    alpha=0.1,
    beta=3.2,
)

# Generating N random samples
N = 2000
dataset = sim.generate_data(num_samples=N, num_features=3)

# Showing a few data-points
dataset.head(2)

#### 3 - Creating the modeling dataset
# Defining the features
features = sim.features

# Building training and testing sets #
index_train, index_test = train_test_split(range(N), test_size=0.2)
data_train = dataset.loc[index_train].reset_index(drop=True)
data_test = dataset.loc[index_test].reset_index(drop=True)

# Creating the X, T and E input
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train["time"].values, data_test["time"].values
E_train, E_test = data_train["event"].values, data_test["event"].values
X_valid, T_valid, E_valid = X_test, T_test, E_test

#### 4 - Creating an instance of the NonLinear CoxPH model and fitting
# the data.

# Defining the MLP structure. Here we will build a 1-hidden layer
# with 150 units and `BentIdentity` as its activation function
structure = [
    {"activation": "BentIdentity", "num_units": 150},
]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Building the model
nonlinear_coxph = MineNonLinearCoxPHModel(structure=structure, auto_scaler=False)
nonlinear_coxph.fit(
    X_train,
    T_train,
    E_train,
    X_valid,
    T_valid,
    E_valid,
    lr=1e-3,
    init_method="xav_uniform",
)


#### 5 - Cross Validation / Model Performances
c_index = concordance_index(nonlinear_coxph, X_test, T_test, E_test)
print("C-index: {:.2f}".format(c_index))

ibs = integrated_brier_score(
    nonlinear_coxph, X_test, T_test, E_test, t_max=10, figure_size=(20, 6.5)
)

Number of data-points: 2000 - Number of events: 1062.0
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863


[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.4139838857585

[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.4139838857585

[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.4139838857585

[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.4139838857585

...

[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.4139838857585

[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.4139838857585

[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.4139838857585

[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863
[0.8901786665732414, 1.1008199996734895, 1.182093560165321, 1.5394308489109898, 1.9780214359537456, 2.413983885758536, 2.640328855306041, 2.9561555868679683]
3.5459648966172863


KeyboardInterrupt: 

In [ ]:
from pysurvival_mine.utils.display import compare_to_actual

In [ ]:
compare_to_actual(
    nonlinear_coxph,
    X_test,
    T_test,
    E_test,
    is_at_risk=True,
    figure_size=(16, 6),
    metrics=["rmse", "mean", "median"],
)

In [ ]:
plt.figure()
plt.plot(nonlinear_coxph.metrics["c_index_valid"], label="valid")
plt.plot(nonlinear_coxph.metrics["c_index_train"], label="train")